<a href="https://colab.research.google.com/github/ATLAS097/Data-Engineering-CA2/blob/Swam/data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Cleaning
---
### Date: 21 July 2025
### Class: DAAA/FT/2B/22
### Group-5
### Group Member: Kaung Myat San, Swam Htet Aung, Ethan Ng Wen Xian
---

## Loading Modules

In [2]:
!pip install python-docx
import re
import pandas as pd
from docx import Document
import warnings
warnings.filterwarnings('ignore')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.0 MB/s eta 0:00:00


## Loading Data
---

In [62]:
# read appointment.xlsx from a URL (must use the raw file link and specify engine)
appointment = pd.read_excel(
	"https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/appointment.xlsx",
	engine="openpyxl"
)

In [9]:
!wget https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/facility.docx -O facility.docx
document = Document("facility.docx")

table = document.tables[0]

data = []
for row in table.rows:
    data.append([cell.text for cell in row.cells])

# Convert to pandas DataFrame
facility = pd.DataFrame(data)

# Set the first row as header and remove it from data
facility.columns = facility.iloc[0]
facility = facility[1:].reset_index(drop=True)
# rename BuldingID to BuildingID
facility.rename(columns={'BuldingID': 'BuildingID'}, inplace=True)

display(facility.head())

--2025-08-08 05:28:35--  https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/facility.docx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34082 (33K) [application/octet-stream]
Saving to: ‘facility.docx’

facility.docx       100%[===================>]  33.28K  --.-KB/s    in 0.01s   

2025-08-08 05:28:35 (3.27 MB/s) - ‘facility.docx’ saved [34082/34082]



,Department,BuildingID,BuildingName,RoomID,Purpose
0,Emergency,B0001,Emergency 1,R0101,Consultation Room
1,Emergency,B0001,Emergency 1,R0102,Consultation Room
2,Emergency,B0001,Emergency 1,R0103,Emergency Room
3,Emergency,B0001,Emergency 1,R0104,Pharmacy
4,Emergency,B0001,Emergency 1,R0201,Laboratory


In [10]:
patient = pd.read_excel("https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/patient.xlsx", engine="openpyxl")
patient.head()

,PatientID,Name,Sex,Contact,email,Address,DOB
0,P0001,Kelsey Moore,female,39472495,kelsey.moore@gmail.com,"34892 Christopher Crossing, South Brooke, NH 1...",1997-05-05
1,P0002,William Perry,male,15351117,william.perry@hotmail.com,"94501 Bell Islands Suite 485, New Jessica, NJ ...",2007-06-06
2,P0003,Nicholas Jones,male,56648257,nicholas.jones@yahoo.com,"8841 Lewis Parkways Apt. 040, South Kristen, I...",1975-12-04
3,P0004,Theresa Smith,female,45807263,theresa.smith@outlook.com,"271 Jeffrey Freeway, Willisbury, ME 39487",1997-09-14
4,P0005,Tara Kennedy,female,84234563,tara.kennedy@protonmail.com,"64128 Rick Coves Apt. 284, Lake James, NJ 51964",2011-02-22


In [61]:
staff = pd.read_csv("https://raw.githubusercontent.com/ATLAS097/Data-Engineering-CA2/main/data/staff.csv")
staff.head()

,DoctorID,Name,Contact,Email,DepartmentID,Department
0,E0001,Grace Chen,78901234,grace.chen@abchospital.com,D0005,Pediatrics
1,E0002,Henry Adams,75678901,henry.adams@abchospital.com,D0002,Intensive Care Unit
2,E0003,Maya Patel,44567890,maya.patel@abchospital.com,D0008,Surgery
3,E0004,Sean Underwood,84345678,sean.underwood@abchospital.com,D0001,Emergency
4,E0005,Amy Bryant,72123456,amy.bryant@abchospital.com,D0009,Neurology


Let us first examine the count of rows for each dataset.

In [7]:
print("Number of rows in the appointment DataFrame:", len(appointment))
print("Number of rows in the facility DataFrame:", len(facility))
print("Number of rows in the patient DataFrame:", len(patient))
print("Number of rows in the staff DataFrame:", len(staff))

Number of rows in the appointment DataFrame: 100001
Number of rows in the facility DataFrame: 49
Number of rows in the patient DataFrame: 1000
Number of rows in the staff DataFrame: 100


# Facility Data Analysis and Cleaning
---

In [13]:
facility.head()

,Department,BuildingID,BuildingName,RoomID,Purpose
0,Emergency,B0001,Emergency 1,R0101,Consultation Room
1,Emergency,B0001,Emergency 1,R0102,Consultation Room
2,Emergency,B0001,Emergency 1,R0103,Emergency Room
3,Emergency,B0001,Emergency 1,R0104,Pharmacy
4,Emergency,B0001,Emergency 1,R0201,Laboratory


In [17]:
facility.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Department    49 non-null     object
 1   BuildingID    49 non-null     object
 2   BuildingName  49 non-null     object
 3   RoomID        49 non-null     object
 4   Purpose       49 non-null     object
dtypes: object(5)
memory usage: 2.0+ KB


It seems all the columns are of correct data types and free of null values.

## Check for Duplicate Rows

We will search for the duplicated rows, first.

In [15]:
duplicate_rows = facility.duplicated()
num_duplicates = duplicate_rows.sum()

print(f"\nNumber of duplicate rows found: {num_duplicates}")

# If duplicates exist, display them
if num_duplicates > 0:
    print("\nHere are the duplicate rows:")
    # The 'keep=False' argument shows all occurrences of duplicates
    print(facility[facility.duplicated(keep=False)])
else:
    print("There are no duplicate rows in the dataset.")


Number of duplicate rows found: 0
There are no duplicate rows in the dataset.


There are no completely duplicate rows in the dataset. However, since the combination of `BuildingID`, `BuildingName`, and `RoomID` serves as the primary key, we should check for duplicate records based on these columns to ensure data integrity.

In [ ]:
# Check for duplicate rows based on 'BuildingID', 'BuildingName', and 'RoomID'
duplicate_facility_subset = facility.duplicated(subset=['BuildingID', 'BuildingName', 'RoomID'], keep=False)

# Filter the DataFrame to show only the duplicate rows
duplicate_facility_rows = facility[duplicate_facility_subset]

print("Rows with duplicate 'BuildingID', 'BuildingName', and 'RoomID' combinations:")
if not duplicate_facility_rows.empty:
    display(duplicate_facility_rows)
else:
    print("No duplicate combinations of 'BuildingID', 'BuildingName', and 'RoomID' found.")



Rows with duplicate 'BuildingID', 'BuildingName', and 'RoomID' combinations:
No duplicate combinations of 'BuildingID', 'BuildingName', and 'RoomID' found.


There are no duplicate rows in the dataset, even when considering the combination of `BuildingID`, `BuildingName`, and `RoomID` as the primary key.





## Checking for Nulls

In [18]:
# Check for null values in each column
null_counts = facility.isnull().sum()

# Display columns with at least one null value
null_counts = null_counts[null_counts > 0]

print("Columns with null values:")
if not null_counts.empty:
    print(null_counts)
else:
    print("No null values found.")


Columns with null values:
No null values found.


There are no null values.

## Checking for Categorial columns

In [21]:
print("Unique Values in Each Column")

for column in facility.columns:
    print(f"\n Column: '{column}'")

    # Get and print the unique values
    unique_values = facility[column].unique()
    print(unique_values)

Unique Values in Each Column

 Column: 'Department'
['Emergency' 'Cardiology' 'Pediatrics' 'Orthopedics'
 'Obstetrics and Gynecology' 'Surgery' 'Neurology' 'Oncology' 'Radiology'
 'Intensive Care Unit' '']

 Column: 'BuildingID'
['B0001 ' 'B0002 ' 'B0003 ' 'B0004 ' '']

 Column: 'BuildingName'
['Emergency 1' 'Central Tower' 'ICU Tower' 'Emergency 2' '']

 Column: 'RoomID'
[' R0101' ' R0102' ' R0103' ' R0104' ' R0201' ' R0202' ' R0203' ' R0204'
 ' R0301' ' R0302' ' R0303' ' R0304' ' R0401' ' R0402' ' R0403' ' R0404'
 '']

 Column: 'Purpose'
['Consultation Room' 'Emergency Room' 'Pharmacy' 'Laboratory' 'X-ray Room'
 'Operating Room' 'Treatment Room' 'Critical Care Room'
 'Isolation ICU Room' 'MRI Room' '']


We observe that the empty string `''` appears as last entry  in every column. We will investigate this further to understand its implications in the dataset. We will check last few columns.

In [22]:
facility.tail()

,Department,BuildingID,BuildingName,RoomID,Purpose
44,Pediatrics,B0004,Emergency 2,R0201,Treatment Room
45,Obstetrics and Gynecology,B0004,Emergency 2,R0202,Consultation Room
46,Obstetrics and Gynecology,B0004,Emergency 2,R0203,Consultation Room
47,Radiology,B0004,Emergency 2,R0204,X-ray Room
48,,,,,


The last row contains only empty values in all columns, so we will remove it from the dataset.

In [23]:
facility = facility.iloc[:-1]

Let's check if this row has been removed.

In [29]:
print("Unique Values in Each Column")

for column in facility.columns:
    print(f"\n Column: '{column}'")

    # Get and print the unique values
    unique_values = facility[column].unique()
    print(unique_values)

Unique Values in Each Column

 Column: 'Department'
['Emergency' 'Cardiology' 'Pediatrics' 'Orthopedics'
 'Obstetrics and Gynecology' 'Surgery' 'Neurology' 'Oncology' 'Radiology'
 'Intensive Care Unit']

 Column: 'BuildingID'
['B0001' 'B0002' 'B0003' 'B0004']

 Column: 'BuildingName'
['Emergency 1' 'Central Tower' 'ICU Tower' 'Emergency 2']

 Column: 'RoomID'
['R0101' 'R0102' 'R0103' 'R0104' 'R0201' 'R0202' 'R0203' 'R0204' 'R0301'
 'R0302' 'R0303' 'R0304' 'R0401' 'R0402' 'R0403' 'R0404']

 Column: 'Purpose'
['Consultation Room' 'Emergency Room' 'Pharmacy' 'Laboratory' 'X-ray Room'
 'Operating Room' 'Treatment Room' 'Critical Care Room'
 'Isolation ICU Room' 'MRI Room']


## Checking for White spaces

In [28]:
# Check for leading/trailing whitespace in each string column
for column in facility.columns:
    if facility[column].dtype == 'object':
        has_whitespace = facility[column].astype(str) != facility[column].astype(str).str.strip()
        result = "Yes" if has_whitespace.any() else "No"
        print(f"Column '{column}': {result}")



Column 'Department': No
Column 'BuildingID': No
Column 'BuildingName': No
Column 'RoomID': No
Column 'Purpose': No


Since two columns contain whitespace characters, we will remove the whitespace from these columns to ensure data consistency.

In [27]:
# Clean leading/trailing whitespaces and count affected rows
columns_to_clean = ['BuildingID', 'RoomID']

for col in columns_to_clean:
    # Count rows with whitespace before cleaning
    before_clean = facility[col].astype(str)
    has_whitespace = before_clean != before_clean.str.strip()
    affected_rows = has_whitespace.sum()

    # Clean the column
    facility[col] = before_clean.str.strip()

    print(f"Whitespace removed from column '{col}': {affected_rows} row(s) affected.")


Whitespace removed from column 'BuildingID': 48 row(s) affected.
Whitespace removed from column 'RoomID': 48 row(s) affected.


# Appointment Data Analysis and Cleaning
---

In [30]:
appointment.head()

,PatientID,DoctorID,BuildingID,RoomID,ScheduledStart,ScheduledEnd,Arrival,ActualStart,ActualEnd,Weight(kg),Temperature(celcius),Comment
0,P0191,E0066,B0001,R0402,2024-01-05 14:00:00,2024-01-05 15:00:00,2024-01-05 14:35:00,2024-01-05 14:35:00,2024-01-05 15:11:00,49.5,37.2,Orientation comfortably
1,P0776,E0065,B0004,R0104,2024-02-27 14:00:00,2024-02-27 15:00:00,2024-02-27 14:59:00,2024-02-27 14:59:00,2024-02-27 16:03:00,55.5,37.1,Imaging has improved slightly
2,P0157,E0051,B0001,R0301,2022-12-28 09:00:00,2022-12-28 10:00:00,NaT,NaT,NaN,NaN,NaN,NaN
3,P0593,E0044,B0004,R0101,2023-01-04 15:00:00,2023-01-04 16:00:00,2023-01-04 15:48:00,2023-01-04 15:48:00,2023-01-04 16:22:00,50.5,36.8,Rash reported today
4,P0434,E0074,B0002,R0203,2020-04-14 09:00:00,2020-04-14 10:00:00,2020-04-14 09:39:00,2020-04-14 09:39:00,2020-04-14 10:07:00,49.5,38.1,Continue observed on forearm


In [37]:
appointment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   PatientID             100000 non-null  object        
 1   DoctorID              100000 non-null  object        
 2   BuildingID            100000 non-null  object        
 3   RoomID                100000 non-null  object        
 4   ScheduledStart        100000 non-null  datetime64[ns]
 5   ScheduledEnd          100000 non-null  datetime64[ns]
 6   Arrival               97952 non-null   datetime64[ns]
 7   ActualStart           97952 non-null   datetime64[ns]
 8   ActualEnd             97952 non-null   object        
 9   Weight(kg)            97952 non-null   float64       
 10  Temperature(celcius)  97952 non-null   float64       
 11  Comment               97952 non-null   object        
dtypes: datetime64[ns](4), float64(2), object(6)
memory usage: 9.9+ M

Data types are correct, but there are null values.

## Checking for Duplicate Rows

In [104]:
duplicate_rows = appointment.duplicated()
num_duplicates = duplicate_rows.sum()

print(f"\nNumber of duplicate rows found: {num_duplicates}")

# If duplicates exist, display them
if num_duplicates > 0:
    print("\nHere are the duplicate rows:")
    # The 'keep=False' argument shows all occurrences of duplicates
    print(appointment[appointment.duplicated(keep=False)])
else:
    print("There are no duplicate rows in the dataset.")


Number of duplicate rows found: 0
There are no duplicate rows in the dataset.


## Checking for Nulls


In [68]:
# Check for null values in each column of the appointment DataFrame
null_counts = appointment.isnull().sum()

# Display columns with at least one null value
null_counts = null_counts[null_counts > 0]

print("Columns with null values in 'appointment':")
if not null_counts.empty:
    print(null_counts)
else:
    print("No null values found in 'appointment'.")


Columns with null values in 'appointment':
PatientID                  1
DoctorID                   1
BuildingID                 1
RoomID                     1
ScheduledStart             1
ScheduledEnd               1
Arrival                 2049
ActualStart             2049
ActualEnd               2048
Weight(kg)              2048
Temperature(celcius)    2048
Comment                 2049
dtype: int64




1. Business Scenario

- **No-show patients**: For patients who booked an appointment but never arrived, it is expected that all columns from **"Arrival"** onwards (e.g., `Arrival`, `ActualStart`, `ActualEnd`, etc.) will be **NULL**.
- **Scheduled appointment fields**: The following columns should **never be NULL** for any appointment record:
  - `PatientID`
  - `DoctorID`
  - `BuildingID`
  - `RoomID`
  - `ScheduledStart`
  - `ScheduledEnd`

---

2. Anomaly Detected

- There is **one row** found where all of the critical scheduling fields (`PatientID`, `DoctorID`, `BuildingID`, `RoomID`, `ScheduledStart`, `ScheduledEnd`) are **NULL**.
  - **Expected behavior:** These fields should always have values, as they are required to define any appointment.
  - **Action Item:** We have to investigate the source of this row.

---

3. Inconsistent NULL Counts

- **Observation:** The number of NULL values for columns starting from **"Arrival"** onwards are **not equal**.
  - **Expected behavior:** For every no-show patient, all columns from "Arrival" onwards should be NULL, resulting in matching NULL counts across these columns.
  - **Issue:** Mismatched NULL counts suggest data inconsistency, which we have to examine further.

---

4. Summary Table

| Column                 | Can be NULL for No-Shows? |
|------------------------|:------------------------:|
| PatientID              |            No            |
| DoctorID               |            No            |
| BuildingID             |            No            |
| RoomID                 |            No            |
| ScheduledStart         |            No            |
| ScheduledEnd           |            No            |
| Arrival                |           Yes            |
| ActualStart            |           Yes            |
| ActualEnd              |           Yes            |
| Weight (kg)            |           Yes            |
| Temperature (celsius)  |           Yes            |
| Comment                |           Yes            |


---




In [69]:
# Define the columns to check
cols = ['Arrival', 'ActualStart', 'ActualEnd', 'Weight(kg)', 'Temperature(celcius)', 'Comment']

# Rows where all of these columns have values (completely filled)
not_null_rows = appointment[appointment[cols].notnull().all(axis=1)]
print("Rows where all specified columns have values:", len(not_null_rows))

# Rows where all of these columns are completely empty (null)
selected_rows_null_rows = appointment[appointment[cols].isnull().all(axis=1)]
print("Rows where all specified columns are missing (null):", len(selected_rows_null_rows))

# Rows where some columns are filled and some are missing (partially filled)
mixed_rows = appointment[
    ~(appointment[cols].isnull().all(axis=1)) &
    ~(appointment[cols].notnull().all(axis=1))
]
print("Rows where some of the specified columns are filled and some are missing:", len(mixed_rows))

# Show those mixed rows
appointment[appointment.index.isin(mixed_rows.index)]


Rows where all specified columns have values: 97952
Rows where all specified columns are missing (null): 2048
Rows where some of the specified columns are filled and some are missing: 1


,PatientID,DoctorID,BuildingID,RoomID,ScheduledStart,ScheduledEnd,Arrival,ActualStart,ActualEnd,Weight(kg),Temperature(celcius),Comment
100000,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,average,51.497356,37.140657,NaN


From the analysis above, we observed there is a row with missing data for columns that should never be NULL (such as `PatientID`, `DoctorID`, `BuildingID`, `RoomID`, `ScheduledStart`, and `ScheduledEnd`). This row also contains nulls for some of the specified rows and non-null values for some. Since this row does not conform to the expected data structure and cannot be interpreted meaningfully, **we will remove this anomalous row from the data set** to maintain data quality and integrity.


In [70]:
# remove rows where index is in mixed_rows
appointment = appointment[~appointment.index.isin(mixed_rows.index)]


Let us check nulls again.

In [71]:
# Check for null values in each column of the appointment DataFrame
null_counts = appointment.isnull().sum()

# Display columns with at least one null value
null_counts = null_counts[null_counts > 0]

print("Columns with null values in 'appointment':")
if not null_counts.empty:
    print(null_counts)
else:
    print("No null values found in 'appointment'.")

Columns with null values in 'appointment':
Arrival                 2048
ActualStart             2048
ActualEnd               2048
Weight(kg)              2048
Temperature(celcius)    2048
Comment                 2048
dtype: int64


We have successfully removed the row.

## Checking for White spaces

In [72]:
# Check for leading/trailing whitespace in each string column of the appointment DataFrame
for column in appointment.columns:
    if appointment[column].dtype == 'object':
        has_whitespace = appointment[column].astype(str) != appointment[column].astype(str).str.strip()
        result = "Yes" if has_whitespace.any() else "No"
        print(f"Column '{column}': {result}")


Column 'PatientID': No
Column 'DoctorID': No
Column 'BuildingID': No
Column 'RoomID': No
Column 'ActualEnd': No
Column 'Comment': No


## Checking with other datasets
---


We will clean the white spaces first before comparing.

In [96]:
# --- Forward Checks ---
print("=== Forward Checks (Appointment -> Supporting Datasets) ===")

# Check if PatientID in appointment exists in patient
patient_id_check = appointment['PatientID'].isin(patient['PatientID']).all()
print(f"All PatientIDs in appointment are in patient dataset: {patient_id_check}")

# Check if DoctorID in appointment exists in staff
doctor_id_check = appointment['DoctorID'].isin(staff['DoctorID']).all()
print(f"All DoctorIDs in appointment are in staff dataset: {doctor_id_check}")

# Check if BuildingID in appointment exists in facility
building_id_check = appointment['BuildingID'].isin(facility['BuildingID']).all()
print(f"All BuildingIDs in appointment are in facility dataset: {building_id_check}")

# Check if RoomID in appointment exists in facility
room_id_check = appointment['RoomID'].isin(facility['RoomID']).all()
print(f"All RoomIDs in appointment are in facility dataset: {room_id_check}")

# Check if the combined BuildingID and RoomID from appointment exist in facility
appointment['BuildingRoom'] = appointment['BuildingID'] + appointment['RoomID']
facility['BuildingRoom'] = facility['BuildingID'] + facility['RoomID']
building_room_check = appointment['BuildingRoom'].isin(facility['BuildingRoom']).all()
print(f"All BuildingID and RoomID combinations in appointment are in facility: {building_room_check}")

# --- Reverse Checks ---
print("\n=== Reverse Checks (Supporting Datasets -> Appointment) ===")

# Check if all PatientIDs in patient are used in appointment
patient_id_reverse_check = patient['PatientID'].isin(appointment['PatientID']).all()
print(f"All PatientIDs in patient dataset are used in appointment: {patient_id_reverse_check}")

# Check if all DoctorIDs in staff are used in appointment
doctor_id_reverse_check = staff['DoctorID'].isin(appointment['DoctorID']).all()
print(f"All DoctorIDs in staff dataset are used in appointment: {doctor_id_reverse_check}")

# Check if all BuildingIDs in facility are used in appointment
building_id_reverse_check = facility['BuildingID'].isin(appointment['BuildingID']).all()
print(f"All BuildingIDs in facility are used in appointment: {building_id_reverse_check}")

# Check if all RoomIDs in facility are used in appointment
room_id_reverse_check = facility['RoomID'].isin(appointment['RoomID']).all()
print(f"All RoomIDs in facility are used in appointment: {room_id_reverse_check}")

# Check if all BuildingRoom combinations in facility are used in appointment
building_room_reverse_check = facility['BuildingRoom'].isin(appointment['BuildingRoom']).all()
print(f"All BuildingID and RoomID combinations in facility are used in appointment: {building_room_reverse_check}")

# Drop the temporary combined columns
appointment.drop(columns=['BuildingRoom'], inplace=True)
facility.drop(columns=['BuildingRoom'], inplace=True)


=== Forward Checks (Appointment -> Supporting Datasets) ===
All PatientIDs in appointment are in patient dataset: True
All DoctorIDs in appointment are in staff dataset: True
All BuildingIDs in appointment are in facility dataset: False
All RoomIDs in appointment are in facility dataset: True
All BuildingID and RoomID combinations in appointment are in facility: False

=== Reverse Checks (Supporting Datasets -> Appointment) ===
All PatientIDs in patient dataset are used in appointment: True
All DoctorIDs in staff dataset are used in appointment: False
All BuildingIDs in facility are used in appointment: False
All RoomIDs in facility are used in appointment: False
All BuildingID and RoomID combinations in facility are used in appointment: False


Based on our referential integrity checks, we found mismatches between datasets — for example, some DoctorID, BuildingID, and RoomID values are not consistently present across all datasets.
To retain as much data as possible, we might need to combine records from both sides rather than strictly filtering based on one primary dataset.

While the appointment dataset has the largest number of rows (100,000), our goal is to maximize usable records by merging intelligently and keeping entries that have valid references in either direction wherever appropriate.


* **DoctorID** → Use `staff` dataset
* **PatientID** → Can use either `appointment` or `patient` datasets
* **BuildingID** → Further examination needed
* **RoomID** → Use `facility` dataset


In [93]:
# Filter rows where BuildingID in appointment is NOT in facility
building_id_only_in_appointment = appointment[~appointment['BuildingID'].isin(facility['BuildingID'])]

# Display full rows
print("\nBuildingIDs found only in appointment, not in facility:")
print(building_id_only_in_appointment)




BuildingIDs found only in appointment, not in facility:
      PatientID DoctorID BuildingID RoomID      ScheduledStart  \
44299     P0029    E0015      BB002  R0203 2020-09-13 09:00:00   

             ScheduledEnd             Arrival         ActualStart  \
44299 2020-09-13 10:00:00 2020-09-13 09:16:00 2020-09-13 09:16:00   

                 ActualEnd  Weight(kg)  Temperature(celcius)  \
44299  2020-09-13 09:38:00        50.5                  38.6   

                           Comment  
44299  Mobility shows no discharge  


This is most likely be inconsistent data. We believe that BB002 should be B0002, but we will examine further.

In [98]:
# Create combined key for checking
appointment['BuildingRoom'] = appointment['BuildingID'] + appointment['RoomID']
facility['BuildingRoom'] = facility['BuildingID'] + facility['RoomID']

# Check: appointment → facility
missing_in_facility = appointment[~appointment['BuildingRoom'].isin(facility['BuildingRoom'])]
print("Rows in 'appointment' where BuildingID + RoomID is not found in 'facility':")
print(missing_in_facility)

# Drop temporary columns
appointment.drop(columns=['BuildingRoom'], inplace=True)
facility.drop(columns=['BuildingRoom'], inplace=True)



Rows in 'appointment' where BuildingID + RoomID is not found in 'facility':
      PatientID DoctorID BuildingID RoomID      ScheduledStart  \
44299     P0029    E0015      BB002  R0203 2020-09-13 09:00:00   

             ScheduledEnd             Arrival         ActualStart  \
44299 2020-09-13 10:00:00 2020-09-13 09:16:00 2020-09-13 09:16:00   

                 ActualEnd  Weight(kg)  Temperature(celcius)  \
44299  2020-09-13 09:38:00        50.5                  38.6   

                           Comment BuildingRoom  
44299  Mobility shows no discharge   BB002R0203  


Now we know the reason why
- All BuildingIDs in appointment are in facility dataset: False
- All BuildingID and RoomID combinations in appointment are in facility: False

- It seems it is beacuse of spelling error where `BBOO2` was inserted instead of `B0002`.
- Therefore, we will change it.

In [100]:
# Correct the BuildingID for the specific row
appointment.loc[44299, 'BuildingID'] = 'B0002'


In [101]:
# Check if BuildingID in appointment exists in facility
building_id_check = appointment['BuildingID'].isin(facility['BuildingID']).all()
print(f"All BuildingIDs in appointment are in facility dataset: {building_id_check}")

# Check if combined BuildingID + RoomID exists in facility
appointment['BuildingRoom'] = appointment['BuildingID'] + appointment['RoomID']
facility['BuildingRoom'] = facility['BuildingID'] + facility['RoomID']

building_room_check = appointment['BuildingRoom'].isin(facility['BuildingRoom']).all()
print(f"All BuildingID and RoomID combinations in appointment are in facility: {building_room_check}")

# Drop temporary columns
appointment.drop(columns=['BuildingRoom'], inplace=True)
facility.drop(columns=['BuildingRoom'], inplace=True)


All BuildingIDs in appointment are in facility dataset: True
All BuildingID and RoomID combinations in appointment are in facility: True


Now, we have verified that facility dataset contains all the buildingId and all the combinations of building id and room id.

## Checking for date/time columns
---

In [102]:
# Convert 'ActualEnd' to datetime, coercing errors
appointment['ActualEnd'] = pd.to_datetime(appointment['ActualEnd'], errors='coerce')


# Check for dates outside a reasonable range
# Determine a reasonable date range based on ScheduledStart and ScheduledEnd
min_scheduled_date = appointment['ScheduledStart'].min()
max_scheduled_date = appointment['ScheduledEnd'].max()

print(f"Earliest Scheduled Start Date: {min_scheduled_date}")
print(f"Latest Scheduled End Date: {max_scheduled_date}")

# Define a buffer period (e.g., 1 year before the earliest scheduled start and 1 year after the latest scheduled end)
date_buffer = pd.DateOffset(years=1)
earliest_reasonable_date = min_scheduled_date - date_buffer
latest_reasonable_date = max_scheduled_date + date_buffer

date_columns = ['ScheduledStart', 'ScheduledEnd', 'Arrival', 'ActualStart', 'ActualEnd']

for col in date_columns:
    # Check for dates before the earliest reasonable date
    invalid_earlier_dates = appointment[appointment[col] < earliest_reasonable_date]
    if not invalid_earlier_dates.empty:
        print(f"\nRows with {col} before {earliest_reasonable_date.date()}:")
        display(invalid_earlier_dates)

    # Check for dates after the latest reasonable date
    invalid_later_dates = appointment[appointment[col] > latest_reasonable_date]
    if not invalid_later_dates.empty:
        print(f"\nRows with {col} after {latest_reasonable_date.date()}:")
        display(invalid_later_dates)

    # Check for NaT values (already done by .info(), but can explicitly check if needed)
    nat_count = appointment[col].isnull().sum()
    if nat_count > 0:
        print(f"\nNumber of NaT values in {col}: {nat_count}")

# Check if ScheduledEnd is before ScheduledStart
invalid_scheduled_times = appointment[appointment['ScheduledEnd'] < appointment['ScheduledStart']]
if not invalid_scheduled_times.empty:
    print("\nRows where ScheduledEnd is before ScheduledStart:")
    display(invalid_scheduled_times)
else:
    print("\nAll ScheduledEnd dates are after ScheduledStart dates.")

# Check if ActualStart is before Arrival
invalid_arrival_times = appointment[appointment['ActualStart'] < appointment['Arrival']]
if not invalid_arrival_times.empty:
    print("\nRows where ActualStart is before Arrival:")
    display(invalid_arrival_times)
else:
    print("\nAll ActualStart dates are after Arrival dates.")


# Check if ActualEnd is before ActualStart (invalid)
invalid_actual_times = appointment[appointment['ActualEnd'] < appointment['ActualStart']]
if not invalid_actual_times.empty:
    print("\nRows where ActualEnd is before ActualStart:")
    display(invalid_actual_times)
else:
    print("\nAll ActualEnd dates are after ActualStart dates.")

Earliest Scheduled Start Date: 2020-01-01 09:00:00
Latest Scheduled End Date: 2024-12-31 17:00:00

Number of NaT values in Arrival: 2048

Number of NaT values in ActualStart: 2048

Number of NaT values in ActualEnd: 2048

All ScheduledEnd dates are after ScheduledStart dates.

All ActualStart dates are after Arrival dates.

All ActualEnd dates are after ActualStart dates.


## Checking for numerical columns
---

In [103]:
# Check data types for Weight(kg) and Temperature(celcius)
print("Data types:")
print(appointment[['Weight(kg)', 'Temperature(celcius)']].dtypes)



# Check for viable weight range (e.g., > 0 and within a reasonable upper limit)
# Define a reasonable range (these are examples and might need adjustment based on context)
min_weight = 0
max_weight = 500 # Assuming weight is in kg, 500 kg is a generous upper limit

invalid_weight = appointment[(appointment['Weight(kg)'] <= min_weight) | (appointment['Weight(kg)'] > max_weight)]
if not invalid_weight.empty:
    print(f"\nRows with Weight(kg) outside the range ({min_weight}kg - {max_weight}kg):")
    display(invalid_weight)
else:
    print(f"\nAll Weight(kg) values are within the viable range ({min_weight}kg - {max_weight}kg).")

# Check for viable temperature range (e.g., typically human body temperature range)
# Define a reasonable range in Celsius
min_temp = 30 # Assuming a very low body temperature
max_temp = 45 # Assuming a very high fever

invalid_temp = appointment[(appointment['Temperature(celcius)'] < min_temp) | (appointment['Temperature(celcius)'] > max_temp)]
if not invalid_temp.empty:
    print(f"\nRows with Temperature(celcius) outside the range ({min_temp}°C - {max_temp}°C):")
    display(invalid_temp)
else:
    print(f"\nAll Temperature(celcius) values are within the viable range ({min_temp}°C - {max_temp}°C).")

# Check for nulls in these columns (already done by .info(), but good to reiterate)
print("\nNull counts for Weight(kg) and Temperature(celcius):")
print(appointment[['Weight(kg)', 'Temperature(celcius)']].isnull().sum())

Data types:
Weight(kg)              float64
Temperature(celcius)    float64
dtype: object

All Weight(kg) values are within the viable range (0kg - 500kg).

All Temperature(celcius) values are within the viable range (30°C - 45°C).

Null counts for Weight(kg) and Temperature(celcius):
Weight(kg)              2048
Temperature(celcius)    2048
dtype: int64


## Checking for "Comment" column
---

In [ ]:
# Check data type of the 'Comment' column
print("Data type of 'Comment' column:")
display(appointment['Comment'].dtype)

# Check for null values in 'Comment'
null_comments_count = appointment['Comment'].isnull().sum()
print(f"\nNumber of null comments: {null_comments_count}")

# Check for non-string data types in 'Comment' (if the dtype is object)
if appointment['Comment'].dtype == 'object':
    non_string_comments = appointment[appointment['Comment'].apply(lambda x: not isinstance(x, str) and pd.notna(x))]
    if not non_string_comments.empty:
        print("\nRows with non-string comments:")
        display(non_string_comments)
    else:
        print("\nAll non-null comments are strings.")

Data type of 'Comment' column:


dtype('O')


Number of null comments: 2048

All non-null comments are strings.


# Patient Data Analysis and Cleaning
---

In [84]:
patient.head()

,PatientID,Name,Sex,Contact,email,Address,DOB
0,P0001,Kelsey Moore,female,39472495,kelsey.moore@gmail.com,"34892 Christopher Crossing, South Brooke, NH 1...",1997-05-05
1,P0002,William Perry,male,15351117,william.perry@hotmail.com,"94501 Bell Islands Suite 485, New Jessica, NJ ...",2007-06-06
2,P0003,Nicholas Jones,male,56648257,nicholas.jones@yahoo.com,"8841 Lewis Parkways Apt. 040, South Kristen, I...",1975-12-04
3,P0004,Theresa Smith,female,45807263,theresa.smith@outlook.com,"271 Jeffrey Freeway, Willisbury, ME 39487",1997-09-14
4,P0005,Tara Kennedy,female,84234563,tara.kennedy@protonmail.com,"64128 Rick Coves Apt. 284, Lake James, NJ 51964",2011-02-22


In [85]:
patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   PatientID  1000 non-null   object        
 1   Name       1000 non-null   object        
 2   Sex        1000 non-null   object        
 3   Contact    1000 non-null   int64         
 4   email      1000 non-null   object        
 5   Address    1000 non-null   object        
 6   DOB        1000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 54.8+ KB


It seems all the columns are of correct data types and free of null values.

## Checking for Duplicate Rows

In [87]:
# # Check for fully duplicate rows in the staff DataFrame
duplicate_rows = patient.duplicated()
num_duplicates = duplicate_rows.sum()

print(f"\nNumber of duplicate rows found in 'patient': {num_duplicates}")

# If duplicates exist, display them
if num_duplicates > 0:
    print("\nHere are the duplicate rows:")
    # Show all occurrences of the duplicates
    print(patient[patient.duplicated(keep=False)])
else:
    print("There are no duplicate rows in the 'patient' dataset.")


Number of duplicate rows found in 'patient': 0
There are no duplicate rows in the 'patient' dataset.


## Checking for Nulls

In [88]:
# Check for null values in each column
null_counts = patient.isnull().sum()

# Display columns with at least one null value
null_counts = null_counts[null_counts > 0]

print("Columns with null values:")
if not null_counts.empty:
    print(null_counts)
else:
    print("No null values found.")

Columns with null values:
No null values found.


## Checking for White spaces

In [89]:
# Check for leading/trailing whitespace in each string column
for column in patient.columns:
    if patient[column].dtype == 'object':
        has_whitespace = patient[column].astype(str) != patient[column].astype(str).str.strip()
        result = "Yes" if has_whitespace.any() else "No"
        print(f"Column '{column}': {result}")

Column 'PatientID': No
Column 'Name': No
Column 'Sex': No
Column 'email': No
Column 'Address': No


All the entries do not contain extra white spaces.

## Checking for Categorical Columns

In [90]:
print("Count of Unique Values in Each Column:")

for column in patient.columns:
    unique_count = patient[column].nunique()
    print(f"Column: '{column}' → {unique_count} unique value(s)")

Count of Unique Values in Each Column:
Column: 'PatientID' → 1000 unique value(s)
Column: 'Name' → 990 unique value(s)
Column: 'Sex' → 2 unique value(s)
Column: 'Contact' → 1000 unique value(s)
Column: 'email' → 1000 unique value(s)
Column: 'Address' → 1000 unique value(s)
Column: 'DOB' → 971 unique value(s)


The result makes sense, since `PatientID` has 1000 unique values.

Let us examine some of the columns further.

In [91]:
# Check unique values in the 'Sex' column of the patient DataFrame
unique_sex_values = patient['Sex'].unique()

print("Unique values in the 'Sex' column of the patient DataFrame:")
print(unique_sex_values)

Unique values in the 'Sex' column of the patient DataFrame:
['female' 'male']


Based on the OLTP given statements, we will change "male" to "m" and "female" to "f".

In [92]:
patient['Sex'] = patient['Sex'].replace({'male': 'm', 'female': 'f'})
display(patient.head())

,PatientID,Name,Sex,Contact,email,Address,DOB
0,P0001,Kelsey Moore,f,39472495,kelsey.moore@gmail.com,"34892 Christopher Crossing, South Brooke, NH 1...",1997-05-05
1,P0002,William Perry,m,15351117,william.perry@hotmail.com,"94501 Bell Islands Suite 485, New Jessica, NJ ...",2007-06-06
2,P0003,Nicholas Jones,m,56648257,nicholas.jones@yahoo.com,"8841 Lewis Parkways Apt. 040, South Kristen, I...",1975-12-04
3,P0004,Theresa Smith,f,45807263,theresa.smith@outlook.com,"271 Jeffrey Freeway, Willisbury, ME 39487",1997-09-14
4,P0005,Tara Kennedy,f,84234563,tara.kennedy@protonmail.com,"64128 Rick Coves Apt. 284, Lake James, NJ 51964",2011-02-22


In [ ]:
# Check for hyphens or dashes in the 'Contact' column
contact_contains_punctuation = patient['Contact'].astype(str).str.contains(r'[-–—]', regex=True)

if contact_contains_punctuation.any():
    print("\nRows in 'Contact' column containing hyphens or dashes:")
    # Display rows where punctuation is found
    display(patient[contact_contains_punctuation])
else:
    print("\n'Contact' column does not contain hyphens or dashes.")


'Contact' column does not contain hyphens or dashes.


In [ ]:
# Check if the 'email' column contains '@'
email_contains_at = patient['email'].str.contains('@', na=False)

if email_contains_at.all():
    print("All email addresses in the 'email' column contain '@'.")
else:
    print("Some email addresses in the 'email' column do not contain '@'.")


All email addresses in the 'email' column contain '@'.


# Staff Data Analysis and Cleaning
---

In [45]:
staff.head()

,DoctorID,Name,Contact,Email,DepartmentID,Department
0,E0001,Grace Chen,78901234,grace.chen@abchospital.com,D0005,Pediatrics
1,E0002,Henry Adams,75678901,henry.adams@abchospital.com,D0002,Intensive Care Unit
2,E0003,Maya Patel,44567890,maya.patel@abchospital.com,D0008,Surgery
3,E0004,Sean Underwood,84345678,sean.underwood@abchospital.com,D0001,Emergency
4,E0005,Amy Bryant,72123456,amy.bryant@abchospital.com,D0009,Neurology


In [46]:
staff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   DoctorID      100 non-null    object
 1   Name          100 non-null    object
 2   Contact       100 non-null    int64 
 3   Email         100 non-null    object
 4   DepartmentID  100 non-null    object
 5   Department    100 non-null    object
dtypes: int64(1), object(5)
memory usage: 4.8+ KB


It seems all the columns are of correct data types and free of null values.

## Checking for Duplicate Rows

In [63]:
# # Check for fully duplicate rows in the staff DataFrame
duplicate_rows = staff.duplicated()
num_duplicates = duplicate_rows.sum()

print(f"\nNumber of duplicate rows found in 'staff': {num_duplicates}")

# If duplicates exist, display them
if num_duplicates > 0:
    print("\nHere are the duplicate rows:")
    # Show all occurrences of the duplicates
    print(staff[staff.duplicated(keep=False)])
else:
    print("There are no duplicate rows in the 'staff' dataset.")



Number of duplicate rows found in 'staff': 0
There are no duplicate rows in the 'staff' dataset.


We have verified that this dataset does not contain any duplicate entries.

## Checking for Nulls

In [64]:
# Check for null values in each column
null_counts = staff.isnull().sum()

# Display columns with at least one null value
null_counts = null_counts[null_counts > 0]

print("Columns with null values:")
if not null_counts.empty:
    print(null_counts)
else:
    print("No null values found.")

Columns with null values:
No null values found.


There are no null values.

## Checking for White spaces

In [65]:
# Check for leading/trailing whitespace in each string column
for column in staff.columns:
    if staff[column].dtype == 'object':
        has_whitespace = staff[column].astype(str) != staff[column].astype(str).str.strip()
        result = "Yes" if has_whitespace.any() else "No"
        print(f"Column '{column}': {result}")

Column 'DoctorID': Yes
Column 'Name': No
Column 'Email': Yes
Column 'DepartmentID': No
Column 'Department': No


We will remove the existing white spaces.

In [66]:
# Clean leading/trailing whitespaces and count affected rows
columns_to_clean = ['DoctorID', 'Email']

for col in columns_to_clean:
    # Count rows with whitespace before cleaning
    before_clean = staff[col].astype(str)
    has_whitespace = before_clean != before_clean.str.strip()
    affected_rows = has_whitespace.sum()

    # Clean the column
    staff[col] = before_clean.str.strip()

    print(f"Whitespace removed from column '{col}': {affected_rows} row(s) affected.")

Whitespace removed from column 'DoctorID': 3 row(s) affected.
Whitespace removed from column 'Email': 99 row(s) affected.


## Checking for Categorical Columns

In [67]:
print("Count of Unique Values in Each Column:")

for column in staff.columns:
    unique_count = staff[column].nunique()
    print(f"Column: '{column}' → {unique_count} unique value(s)")


Count of Unique Values in Each Column:
Column: 'DoctorID' → 100 unique value(s)
Column: 'Name' → 100 unique value(s)
Column: 'Contact' → 100 unique value(s)
Column: 'Email' → 100 unique value(s)
Column: 'DepartmentID' → 10 unique value(s)
Column: 'Department' → 11 unique value(s)


- There are 100 rows, each for a different doctor.
- We will check how many unique `DepartmentID` and `Department` values there are.

In [55]:
# Print unique value counts for DepartmentID and Department columns
columns_to_check = ['DepartmentID', 'Department']

for column in columns_to_check:
    print(f"\nUnique value counts in column '{column}':")
    value_counts = staff[column].value_counts()
    print(value_counts.to_string(header=False))





Unique value counts in column 'DepartmentID':
D0004    15
D0009    12
D0007    12
D0005    11
D0010    10
D0001    10
D0006     9
D0002     8
D0003     7
D0008     6

Unique value counts in column 'Department':
Cardiology                   15
Neurology                    12
Obstetrics and Gynecology    12
Pediatrics                   11
Oncology                     10
Emergency                     9
Orthopedics                   9
Intensive Care Unit           8
Radiology                     7
Surgery                       6
Emmergency                    1


- The counts must match across DepartmentID and Department columns, since they are related; one DepartmentID relates to one Department.
- There is one error in the `Department` column: the value "Emmergency" (which we assume should be "Emergency"). We will examine this further.

In [74]:
# Filter staff dataframe for 'Emergency' and 'Emmergency' departments
emergency_departments = staff[staff['Department'].isin(['Emergency', 'Emmergency'])]

# Display the Department and DepartmentID for these rows
print("Department and DepartmentID for 'Emergency' and 'Emmergency' in Staff dataset:")
display(emergency_departments[['Department', 'DepartmentID']])

Department and DepartmentID for 'Emergency' and 'Emmergency' in Staff dataset:


,Department,DepartmentID
3,Emergency,D0001
10,Emergency,D0001
27,Emergency,D0001
32,Emergency,D0001
40,Emergency,D0001
52,Emergency,D0001
63,Emergency,D0001
73,Emergency,D0001
83,Emmergency,D0001
91,Emergency,D0001


Since both "Emmergency" and "Emergency" have the same exact entry in the column `DepartmentID`, we can confirm that this is inconsistent data, which we have to change.

In [79]:
# Replace 'Emmergency' with 'Emergency' in the 'Department' column of the staff DataFrame
staff['Department'] = staff['Department'].replace('Emmergency', 'Emergency')


Let us check.

In [82]:
# Group by Department and check if the number of unique DepartmentIDs is 1 for each group
department_id_consistency = staff.groupby('Department')['DepartmentID'].nunique()

print("Number of unique DepartmentIDs per Department:")
display(department_id_consistency)

# Check if all departments have only one unique DepartmentID
all_consistent = (department_id_consistency == 1).all()

if all_consistent:
    print("\nDepartmentIDs are consistent for each Department name.")
else:
    print("\nInconsistency found: Some Departments have multiple DepartmentIDs.")
    # Optionally, display the departments with inconsistencies
    inconsistent_departments = department_id_consistency[department_id_consistency > 1]
    print("\nDepartments with inconsistent DepartmentIDs:")
    display(inconsistent_departments)

Number of unique DepartmentIDs per Department:


,DepartmentID
Department,
Cardiology,1
Emergency,1
Intensive Care Unit,1
Neurology,1
Obstetrics and Gynecology,1
Oncology,1
Orthopedics,1
Pediatrics,1
Radiology,1



DepartmentIDs are consistent for each Department name.


## Checking matches against facility dataset

In [81]:
# --- Department Consistency Check ---
print("\nDepartment Consistency Check (Facility vs Staff)")

# Get unique department names from both dataframes
unique_departments_facility = facility['Department'].unique()
unique_departments_staff = staff['Department'].unique()

# Check if the sets of unique department names are the same
departments_match = set(unique_departments_facility) == set(unique_departments_staff)

print(f"Do the unique department names in facility and staff match? {departments_match}")

if not departments_match:
    # Find departments only in facility
    departments_only_in_facility = set(unique_departments_facility) - set(unique_departments_staff)
    if departments_only_in_facility:
        print("\nDepartments found only in facility:")
        print(departments_only_in_facility)

    # Find departments only in staff
    departments_only_in_staff = set(unique_departments_staff) - set(unique_departments_facility)
    if departments_only_in_staff:
        print("\nDepartments found only in staff:")
        print(departments_only_in_staff)


=== Department Consistency Check (Facility vs Staff) ===
Do the unique department names in facility and staff match? True


# Importing cleaned csv


* **DoctorID** → Use `staff` dataset
* **PatientID** → Can use either `appointment` or `patient` datasets
* **BuildingID** → Use `facility` dataset
* **RoomID** → Use `facility` dataset

For `Room` dataset, we need to merge two dataset. So, we will check first.

In [105]:
# Get unique values from both columns
facility_departments = set(facility['Department'].unique())
staff_departments = set(staff['Department'].unique())

# Check if they're the same
are_same = facility_departments == staff_departments
print(f"Are the unique values the same? {are_same}")

Are the unique values the same? True


In [ ]:
# Create department mapping from staff dataset
dept_mapping = staff[['Department', 'DepartmentID']].drop_duplicates()
dept_mapping = dict(zip(dept_mapping['Department'], dept_mapping['DepartmentID']))

# Add DepartmentID to facility dataset
facility['DepartmentID'] = facility['Department'].map(dept_mapping)

# Create room_cleaned dataset
room_cleaned = facility[['BuildingID', 'RoomID', 'Purpose', 'DepartmentID']].copy()


In [ ]:
# Rename columns to match SQL table structure
appointment_cleaned = appointment.rename(columns={
   'ScheduledEnd': 'ScheduledEND',
   'Arrival': 'ArrivalTime',
   'ActualStart': 'StartTime',
   'ActualEnd': 'FinishTime',
   'Weight(kg)': 'Weight',
   'Temperature(celcius)': 'Temperature',
   'Comment': 'DoctorComment'
}).copy()

# Select columns in the correct order for the SQL table
appointment_cleaned = appointment_cleaned[['PatientID', 'DoctorID', 'BuildingID', 'RoomID', 'ScheduledStart', 'ScheduledEND', 'ArrivalTime', 'StartTime', 'FinishTime', 'Weight', 'Temperature', 'DoctorComment']]

print("Appointment dataset cleaned and ready")

Appointment dataset cleaned and ready


In [ ]:
# Patient Table
patient.rename(columns={
    'Name': 'PatientName',
    'email': 'Email'
}, inplace=True)

# Convert Contact to string to match VARCHAR(10) in SQL
patient['Contact'] = patient['Contact'].astype(str)

# Doctor Table
# Rename Name to DoctorName
doctor = staff.copy()
doctor.rename(columns={'Name': 'DoctorName'}, inplace=True)

# Convert Contact to string to match VARCHAR(10)
doctor['Contact'] = doctor['Contact'].astype(str)

# Drop the extra Department column since it's not in the SQL table
if 'Department' in doctor.columns:
  doctor.drop(columns=['Department'], inplace=True)

if 'DepartmentName' in doctor.columns:
  doctor.drop(columns=['DepartmentName'], inplace=True)


# Building Table
# Assuming your DataFrame is named `building`

# Rename typo column 'BuldingID' to 'BuildingID'
facility.rename(columns={'BuldingID': 'BuildingID'}, inplace=True)

# Keep only required columns for SQL table
building = facility[['BuildingID', 'BuildingName']]
building = building.drop_duplicates()

# Department Table
# Rename 'Department' column to 'DepartmentName'
staff.rename(columns={'Department': 'DepartmentName'}, inplace=True)

# Keep only DepartmentID and DepartmentName
department = staff[['DepartmentID', 'DepartmentName']]
department = department.drop_duplicates()

Downloading files all dataset is cleaned

In [ ]:
# Define path to save the new CSV (Downloads folder)
# download_path = os.path.expanduser('~/Downloads/patient.csv')

# file path
patient_file = "cleaned_patient.csv"
doctor_file = "cleaned_doctor.csv"
building_file = "cleaned_building.csv"
department_file = "cleaned_department.csv"
room_file = "cleaned_room.csv"
appointment_file = "cleaned_appointment.csv"

# Save the updated DataFrame to CSV without the index column
patient.to_csv(patient_file, index=False)
doctor.to_csv(doctor_file, index=False)
building.to_csv(building_file, index=False)
department.to_csv(department_file, index=False)
room_cleaned.to_csv(room_file, index=False)
appointment_cleaned.to_csv(appointment_file, index=False)

print(f"Modified patient.csv saved to: \n-{patient_file} \n-{doctor_file} \n-{building_file} \n-{department_file} \n-{room_file} \n-{appointment_file}")
try:
    from google.colab import files
    print("\nClick the links below to download the files (Google Colab):")
    # These might not work if the files are very large or in a complex directory structure
    # You might need to manually download from the file explorer in Colab
    files.download(patient_file)
    files.download(doctor_file)
    files.download(building_file)
    files.download(department_file)
    files.download(room_file)
    files.download(appointment_file)
except ImportError:
    print("\nRunning in a local environment (likely VS Code). Files saved to the current directory.")
    print("You can find the files in your file explorer.")

Modified patient.csv saved to: 
-cleaned_patient.csv 
-cleaned_doctor.csv 
-cleaned_building.csv 
-cleaned_department.csv 
-cleaned_room.csv 
-cleaned_appointment.csv

Click the links below to download the files (Google Colab):


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>